In [3]:
#auto encoder
"""layer,loss,train,evaluate,main
   inferenceの部分がencoder,decoderに変換される"""
import numpy as np
import time,argparse
import tensorflow as tf
import keras
from keras.datasets import mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()
#正規化
x_train=x_train.reshape(-1,784).astype('float32')/255
x_test=x_test.reshape(-1,784).astype('float32')/255
#onehot
y_train=keras.utils.to_categorical(y_train,10)
y_test=keras.utils.to_categorical(y_test,10)
print(x_train.shape)
print(y_train.shape[0])

#parameters
n_encoder_hidden_1=1000
n_encoder_hidden_2=500
n_encoder_hidden_3=250
n_decoder_hidden_1=250
n_decoder_hidden_2=500
n_decoder_hidden_3=1000

training_epochs=10
batch_size=64
display_step=1#コンソールに表示させる回数

(60000, 784)
60000


In [1]:
import tensorflow as tf
import keras
from keras.datasets import mnist
import numpy as np
import time,argparse
#parameters
n_encoder_hidden_1=1000
n_encoder_hidden_2=500
n_encoder_hidden_3=250
n_decoder_hidden_1=250
n_decoder_hidden_2=500
n_decoder_hidden_3=1000

training_epochs=10
batch_size=64
display_step=1#コンソールに表示させる回数
def layer(input,weight_shape,bias_shape,phase_train):
    weight_init=tf.random_normal_initialzier(stddev=(2.0/weight_shape[0])**0.5)
    bias_init=tf.constant_initialzier(value=0)
    
    W=tf.get_variable('W',weight_shape,initializer=weight_init)
    b=tf.get_variable('b',bias_shape,initialzier=bias_init)
    
    #総入力
    logits=tf.matmul(input,W)+b
    return tf.nn.sigmoid(layer_norm_batch(logits,weight_shape[1],phase_train))


Using TensorFlow backend.


In [2]:
#バッチ正規化
from tensorflow.python.ops import control_flow_ops
def layer_batch_norm(x,n_out,phase_train):
    beta_init=tf.constant_initializer(value=0.0,dtype=tf.float32)
    gamma_init=tf.constant_initializer(value=1.0,dtype=tf.float32)
    
    beta=tf.get_variable('beta',[n_out],initializer=beta_init)
    gamma=tf.get_variable('gamma',[n_out],initializer=gamma_init)
    
    batch_mean,batch_var=tf.nn.moments(x,[0],name='moments')
    
    ema=tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op=ema.apply([batch_mean,batch_var])
    ema_mean=ema.average(batch_mean)
    ema_var=ema.average(batch_var)
    
    def mean_var_with_update():
        return tf.control_dependencies(tf.identity(batch_mean),tf.identity(batch_var))
    
    mean,var=control_flow_ops.cond(phase_train,
                                  mean_var_with_update,
                                  lambda:(ema_mean,ema_var))
    x_r=tf.reshape(x,[-1,1,1,n_out])
    normed=tf.nn.batch_norm_with_global_normalization(x_r,mean,var,beta,gamma,1e-3,True)
    
    return tr.reshape(normed,[-1,n_out])

    

In [3]:
def encoder(x,n_code,phase_train):
    with tf.variable_scope("encode"):
        with tf.variable_scope("hidden1"):
            hidden_1=layer(x,
                           [784,n_encoder_hidden_1],
                           [n_encoder_hidden_1],
                           phase_train)
            
        with tf.variable_scope("hidden2"):
            hidden_2=layer(hidden_1,
                          [n_encoder_hidden_1,n_encoder_hidden_2],
                          [n_encoder_hidden_2],
                          phase_train)
        with tf.variable_scope("hidden3"):
            hidden_3=layer(hidden_2,
                          [n_encoder_hidden2,n_encoder_hidden_3],
                          [n_encoder_hidden_3],
                          phase_train)
            
        with tf.variable_scope("code"):
            code=layer(hidden_3,
                       [n_encoder_hidden_3,n_code],
                       [n_code],
                      phase_train)
        return code
    
def decoder(code,n_code,phase_train):
    with tf.variable_scope("decode"):
        with tf.variable_scope("hidden1"):
            hidden_1=layer(code,[n_code,n_decoder_hidden_1],[n_decoder_hidden_1],phase_train)
        with tf.variable_scope("hidden2"):
            hidden_2=layer(hidden_1,[n_decoder_hidden_1,n_decoder_hidden_2],[n_decoder_hidden_2],phase_train)
        with tf.variable_scope("hidden3"):
            hidden_3=layer(hidden_2,[n_decoder_hidden_2,n_decoder_hidden_3],[n_decoder_hidden_3],phase_train)
            
        with tf.variable_scope("output"):
            output=layer(hidden_3,[n_decoder_hidden_3,784],[784],phase_train)
            
        return output
            
        

In [4]:
#loss,train,evaluate
def loss(output,x):
    #l2norm
    l2=tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(output,x)),1))
    train_loss=tf.reduce_mean(l2)
    train_summary_op=tf.summary.scalar("train_cost",train_loss)
    return train_loss,train_summary_op

def train(cost,global_step):
    optimizer=tf.train.AdamOptimizer(lr=0.001,beta1=0.9,beta2=0.999,epsilon=1e-08,
                                    use_locking=False,name='Adam')
    train_op=optimizer.minimize(cost,global_step=global_step)
    return train_op
#?
def image_summary(summary_label,tensor):
    tensor_reshaped=tf.reshape(tensor,[-1,28,28,1])
    return tf.summary.image(summary_label,tensor_reshaped)

def evaluate(output,x):
    with tf.variable_scope("validation"):
        in_im_op=image_summary("input_image",x)
        out_im_op=image_summary("output_image",output)
        l2=tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(output,x,name="val_diff")),1))
        val_loss=tf.reduce_mean(l2)
        val_summary_op=tf.summary.scalar("val_cost",val_loss)
        
        return val_loss,in_im_op,out_im_op,val_summary_op
    
        

In [5]:
def create_batch(x,y,batch_size):
    idx=np.arange(y_train.shape[0])
    
    np.random.shuffle(idx)
    x=x[idx]
    y=y[idx]
    
    for i in range(0,x.shape[0],batch_size):
        yield(x[i:i+batch_size,:],y[i:i+batch_size])

In [6]:
#main
if __name__=='__main__':
    parser=argparse.ArgumentParser(description="Test various optimization strategies")
    parser.add_argument("n_code",type=int)
    args=parser.parse_args()
    n_code=args.n_code
    log_dir="mnist_auto_encoder_hidden={}_logs/".format(n_code)
    #x_train,x_testを書く
    (x_train,y_train),(x_test,y_test)=mnist.load_data()
    x_train=x_train.reshape(-1,784).astype('float32')/255
    x_test=x_test.reshape(-1,784).astype('float32')/255
    
    y_train=keras.utils.to_categorical(y_train,10)
    y_test=keras.utils.to_categorical(y_test,10)
    
    g=tf.Graph()
    
    with g.as_default():
        with tf.variable_scope("autoencoder_model"):
            #mnist data shape 28*28=784
            x=tf.placeholder('float',[None,784])
            phase_train=tf.placeholder(tf.bool)
            
            code=encoder(x,n_code,phase_train)
            output=deocoder(code,n_code,phase_train)
            
            cost,train_summary_op=loss(output,x)
            global_step=tf.Variable(0,name='global_step',trainable=False)
            train_op=train(cost,global_step)
            eval_op,in_im_op,out_im_op,val_summary_op=evaluate(output,x)
            
            summary_op=tf.summary.merge_all()#?
            
            saver=tf.train.Saver()
            sess=tf.Session()
            summary_writer=tf.summary.FileWriter(log_dir,graph=sess.graph)
            
            #初期化
            init_op=tf.global_variables_initializer()
            sess.run(init_op)
            
            #training
            for epoch in range(training_epochs):
                avg_cost=0.0
                batch_gen=create_batch(x_train,y_train,batch_size)
                total_batch=int(x_train.shape[0]/batch_size)
                for idx,(batch_x,batch_y) in enumerate(batch_gen):
                    #fitting
                    _,new_cost,train_summary=sess.run([train_op,cost,train_summary_op],
                                                     feed_dict={x:batch_x,phase_train:True})
                    summary_writer.add_summary(train_summary,sess.run(global_step))
                    #costを計算
                    avg_cost+=new_cost/total_batch
                    
                if epoch%display_step==0:
                    print("epoch:%04d   cost:%.4f"%(epoch+1,avg_cost))
                    
                    summary_writer.add_summary(train_summary,sess.run(global_step))
                    
                    validation_loss,in_im,out_im,val_summary=sess.run([eval_op,in_im_op,out_im_op,val_summary_op],
                                                                     feed_dict={x:x_test,phase_train:False})
                    summary_writer.add_summary(in_im,sess.run(global_step))
                    summary_writer.add_summary(out_im,sess.run(global_step))
                    summary_writer.add_summary(val_summary,sess.run(global_step))
                    
                    print('validation acc:%.3f'%(validation_loss))
                    
                    saver.save(sess,"{}/model-checkpoints-{:04d}".format(log_dir,epoch+1),global_step=global_step)
                    
                print('==optimization finished==')
                
                test_loss=sess.run([eval_op],feed_dict={x:x_test,phase_train:False})
                
                print('test loss:',test_loss)
                    
                    
                    
            
            
    
    

usage: ipykernel_launcher.py [-h] n_code
ipykernel_launcher.py: error: argument n_code: invalid int value: 'C:\\Users\\murata\\AppData\\Roaming\\jupyter\\runtime\\kernel-2307f4ae-9546-4133-bccf-1b0b90eb82d3.json'


SystemExit: 2

D:\python\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
